In [8]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from termcolor import colored

pd.set_option("display.max_rows", None, "display.max_columns", None)
#1. time
#2. dow jones (value and plus minus)
#3. open interest (price, oi, change, implied volitality, trade price) 
#4. Current nifty price 


from selenium import webdriver
from time import sleep
global driver
driver = webdriver.Chrome()
driver.get('https://www1.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbolCode=-10000&symbol=NIFTY&symbol=NIFTY&instrument=-&date=-&segmentLink=17&symbolCount=2&segmentLink=17')
sleep(2)   
content = driver.page_source      
with open("nifty.txt", 'w') as f:
    f.write(content)
sleep(2)
driver.close()

In [9]:
f = open("nifty.txt", "r")
f = f.read()
table = pd.read_html(str(f))
data= table[1]

In [10]:
call_chart =[]
call_oi =[]
call_oi_change =[]
call_vol =[]
call_iv =[]
call_ltp =[]
call_netchg =[]
call_bidqty =[]
call_bid_price =[]
call_ask_price =[]
call_askqty =[]
strike =[]
put_bidqty =[]
put_bid_price =[]
put_ask_price =[]
put_askqty =[]
put_netchg =[]
put_ltp =[]
put_iv =[]
put_vol =[]
put_oi_change =[]
put_oi =[]
put_chart =[]

#####################################################################
data = data.replace("-",0) 

#####################################################################
for i in range(len(data)):
    call_chart.append(data['CALLS',        'Chart'][i])
    call_oi.append(float(data['CALLS',           'OI'][i]))
    call_oi_change.append(float(data['CALLS',   'Chng in OI'][i]))
    call_vol.append(float(data['CALLS',       'Volume'][i]))
    call_iv.append(float(data['CALLS',           'IV'][i]))
    call_ltp.append(float(data['CALLS',          'LTP'][i]))
    call_netchg.append(float(data['CALLS',     'Net Chng'][i]))
    call_bidqty.append(float(data['CALLS',       'BidQty'][i]))
    call_bid_price.append(float(data['CALLS',     'BidPrice'][i]))
    call_ask_price.append(float(data['CALLS',     'AskPrice'][i]))
    call_askqty.append(float(data['CALLS',       'AskQty'][i]))
    strike.append(float(data['Unnamed: 11_level_0', 'Strike Price'][i]))
    put_bidqty.append(float(data['PUTS',       'BidQty'][i]))
    put_bid_price.append(float(data['PUTS',     'BidPrice'][i]))
    put_ask_price.append(float(data['PUTS',     'AskPrice'][i]))
    put_askqty.append(float(data['PUTS',       'AskQty'][i]))
    put_netchg.append(float(data['PUTS',     'Net Chng'][i]))
    put_ltp.append(float(data['PUTS',          'LTP'][i]))
    put_iv.append(float(data['PUTS',           'IV'][i]))
    put_vol.append(float(data['PUTS',       'Volume'][i]))
    put_oi_change.append(float(data['PUTS',   'Chng in OI'][i]))
    put_oi.append(float(data['PUTS',           'OI'][i]))
    put_chart.append(data['PUTS',        'Chart'][i])



In [11]:
main = pd.DataFrame(call_oi,columns={'call_oi'})
main['call_oi_change'] = call_oi_change
main['call_vol'] =call_vol 
main['call_iv'] = call_iv
main['call_ltp'] = call_ltp
main['strike'] = strike
main['put_ltp'] = put_ltp
main['put_iv'] = put_iv
main['put_vol'] = put_vol
main['put_oi_change'] = put_oi_change
main['put_oi'] = put_oi
##################################################################################
put_total = 0
call_total = 0
for i in range(len(main["call_oi"])):
    call_total = call_total + main["call_oi"][i] 

for i in range(len(main["put_oi"])):
    put_total = put_total + main["put_oi"][i] 
    #Point of maximum pain:
################################################################################
options = []
for i in range(len(main)):
    options.append(main['call_oi'][i]+main['put_oi'][i])
options = options[:-1]

max_pain1 = 0
for i in range(len(options)):
    if(options[i] == max(options)):
        max_pain1 = main['strike'][i]

options.remove(max(options))

max_pain2 = 0
for i in range(len(options)):
    if(options[i]==max(options)):
        max_pain2 = main['strike'][i+1]

options.remove(max(options))

max_pain3 = 0
for i in range(len(options)):
    if(options[i]==max(options)):
        max_pain3 = main['strike'][i+1]

##################################################################################
for i in range(len(main)):
    options.append(main['call_oi'][i]+main['put_oi'][i])
options = options[:-1]
####################################
drop_index =[153]
for i in range(len(main)):
    if(i<48 or i>80):
        drop_index.append(i)
main = main.drop(main.index[drop_index])


In [39]:
#nifty 
#highlight current level
url = "https://www.ndtv.com/business/marketdata/domestic-index-nse_nifty" #url
response = requests.get(url)

soup = BeautifulSoup(response.text,'html.parser')

spans = soup.find('span', {'class' : 'senx-up-down'})
nifty = spans.text
#################################################################################
url = "https://www.ndtv.com/business/marketdata/domestic-index-nse_nifty" #url
response = requests.get(url)

soup = BeautifulSoup(response.text,'html.parser')

spans = soup.find('span', {'class' : 'senx-up-down'})
nifty = spans.text
###########################
url = "https://www.ndtv.com/business/marketdata/domestic-index-nse_nifty" #url
response = requests.get(url)

soup = BeautifulSoup(response.text,'html.parser')

nifty_point = soup.find('span', {'class' : 'green-btn'})
for i in nifty_point:     
    for j in i:
        nifty_val = float(j)
    break
nifty_point = nifty_point.text

##################################################################################

if (nifty_val >= 0):
    print("\t\t\t\t\tNifty =",end= "")
    print(colored(nifty,'green'),"\t",colored(nifty_point,'green'))
    print("\t\t\t\t\t-------------------------------")
else:
    print("\t\t\t\t\tNifty =",end= "")
    print(colored(nifty,'red'),"\t",colored(nifty_point,'red'))
    print("\t\t\t\t\t-------------------------------")
    
print("\t\t\t\t\tMaximum pain point 1 = ",end="")
print(colored(max_pain1,'blue'))
print("\t\t\t\t\tMaximum pain point 2 =",max_pain2)
print("\t\t\t\t\tMaximum pain point 3 =",max_pain3)
print("\t\t\t\t\tPUTS/CALLS =",put_total/call_total)
main.style.format({"call_oi": "{:20,.0f}", 
                    "put_oi": "{:20,.0f}", })\
                 .hide_index()\
                 .bar(subset=["call_oi",], color='lightgreen')\
                 .bar(subset=["put_oi"], color='#ff9966')

					Nifty =9859.90 	 306.55(3.21%)
					-------------------------------
					Maximum pain point 1 = 9000.0
					Maximum pain point 2 = 9900.0
					Maximum pain point 3 = 9800.0
					PUTS/CALLS = 1.651194057442482


call_oi,call_oi_change,call_vol,call_iv,call_ltp,strike,put_ltp,put_iv,put_vol,put_oi_change,put_oi
"382,950",-25425,1567,0,1362.25,8500,0.05,0,11391,-288600,"1,449,750"
"2,400",-75,4,0,1315.4,8550,0.05,0,234,-5325,"8,100"
"103,200",-20175,429,0,1261,8600,0.05,0,3121,-94050,"374,625"
"7,425",-225,20,0,1214.2,8650,0.05,0,233,-11775,"16,275"
"66,075",-14325,587,0,1162.3,8700,0.05,0,6480,-194775,"476,250"
"1,875",-75,4,0,1111.05,8750,0.1,0,804,-11325,"24,975"
"89,700",-17250,708,0,1061.2,8800,0.05,0,12432,-230475,"576,825"
"5,700",-675,61,0,940,8850,0.05,0,1024,-20775,"36,825"
"34,425",-18375,454,0,960.45,8900,0.05,0,13909,-400050,"574,350"
"2,475",-675,16,0,915,8950,0.05,0,1433,-31650,"46,575"
